In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# URL of page to be scraped
urlnasa = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(urlnasa)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [3]:
# results are returned as an iterable list
title_results = soup.find_all('div', class_='content_title')
teaser_results = soup.find_all('div', class_='rollover_description_inner')

# Loop through returned title results
for resulttitle in title_results:
    # Error handling
    try:
        # Identify and return title of article
        title = resulttitle.find('a').text
    
    except Exception as e:
        print(e)

for result in teaser_results:
    # Error handling
    try:
        # Identify and return paragraph of article
        paragraph = result.text

    except Exception as e:
        print(e)

In [4]:
#scraping Mars facts table
#________________________
urlfact = "https://space-facts.com/mars/"
facttable = pd.read_html(urlfact)

In [5]:
df = facttable[0]

In [6]:
#df = df.rename(columns={0: "params", 1: "values"})
#df = df.set_index("params")

In [7]:
html_table = df.to_html()

In [8]:
#scraping images
#---------------

urlpic = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/lisastroh/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [9]:
#set up list
hemisphere_image_urls = []

#list of hemispheres to click through on website
Hemispheres = ["Cerberus", "Schiaparelli","Syrtis","Valles"]

#loop through list of hemispheres to click on each one on the website
for hemisphere in Hemispheres:
    #restart on the initial browser page
    browser.visit(urlpic)
    #click on hemisphere link
    browser.links.find_by_partial_text(hemisphere).click()
    #get the html
    page = browser.html
    soup_page = BeautifulSoup(page, 'html.parser')
    #find the title and link in the html
    pictitle = soup_page.find('h2', class_="title").text
    piclink = soup_page.find('div', class_='downloads').li.a["href"]
    
    #put in dictionary
    hemisphere_dict = {
        "title":pictitle,
        "img_url":piclink
    }
    #append to list
    hemisphere_image_urls.append(hemisphere_dict)

In [10]:
browser.quit()